# Map options with px and folium for Dash

In [ ]:
import pandas as pd
investments = pd.read_csv("~/data/mintic_investments_clean.csv")

We remove the departments classified as international

In [ ]:
pd.options.display.max_columns=None
investments.head(10)
investments_1 = investments.copy()
investments_1 = investments[investments.departame_cod != '0']
investments_1.head(5) 

Map with plotly express

we import the shp file from the dane geoportal

In [ ]:
import geopandas as gpd
#all four files must be imported: .shp, .shx, .dbf, .prj of DANE Geoportal

mapa_colombia = gpd.read_file("MGN_DPTO_POLITICO.shp")
print(type(mapa_colombia))
print(mapa_colombia.columns)
print(type(mapa_colombia.geometry))
print(mapa_colombia[['DPTO_CCDGO', 'DPTO_CNMBR']])

In [ ]:
investments_1['departame_nombre'].unique()
dept_dict={'bogota': 'BOGOTÃ, D.C.','antioquia' : 'ANTIOQUIA' , 'valle_del_cauca': 'VALLE DEL CAUCA','cundinamarca':'CUNDINAMARCA','atlantico':'ATLÃNTICO','narinio':'NARIÃO','sucre':'SUCRE','bolivar':'BOLÃVAR','tolima':'TOLIMA','boyaca':'BOYACÃ','risaralda':'RISARALDA','vichada':'VICHADA','santander':'SANTANDER','cordoba':'CÃRDOBA','quindio':'QUINDIO','huila':'HUILA','la_guajira':'LA GUAJIRA','guainia':'GUAINÃA','amazonas':'AMAZONAS','caldas':'CALDAS','cauca':'CAUCA','norte_de_santander':' NORTE DE SANTANDER','guaviare':'GUAVIARE','cesar':'CESAR','meta':'META','caqueta':'CAQUETÃ','choco':'CHOCÃ','putumayo':'PUTUMAYO','magdalena':' MAGDALENA','arauca':'ARAUCA','vaupes':'VAUPÃS','casanare':'CASANARE', 'san_andres':'ARCHIPIÃLAGO DE SAN ANDRÃS, PROVIDENCIA Y SA...'}
investments_1=investments_1[investments_1['departame_nombre']!='internacional']
investments_1['departame_nombre'].unique()
investments_1['depart_nombre_mod']=investments_1['departame_nombre'].replace(dept_dict)
investments_1.head(10)

In [ ]:
geo_map_col = mapa_colombia.to_json()

import plotly.express as px
dff1 = investments_1[investments_1['anio_corte']==2016].groupby(['depart_nombre_mod', 'anio_corte']).sum().reset_index()

#Here we can see the syntax used for the creation of choropleth maps:

px.choropleth_mapbox(dff1,                         #Data
        locations='depart_nombre_mod',featureidkey="properties.DPTO_CNMBR", #Column containing the identifiers used in the GeoJSON file 
        color='inversion',   #Column giving the color intensity of the region
        geojson=mapa_colombia.geometry,                          #The GeoJSON file
        zoom=3,                                   #Zoom
        mapbox_style="carto-positron",            #Mapbox style, for different maps you need a Mapbox account and a token
        center={"lat": 5, "lon": -72}, #Center
        color_continuous_scale="Viridis",         #Color Scheme
        opacity=0.5,                              #Opacity of the map
        )

Map with folium

In [ ]:
import folium
geo_map_col = mapa_colombia.to_json()#geometry.to_json()

m = folium.Map(location=[5, -72], zoom_start=5)

investments_1['inversion_millions'] = investments_1['inversion']/1000000

dff = investments_1[investments_1['anio_corte']==2016].groupby(['departame_cod', 'anio_corte']).sum().reset_index() #2016 map
print(dff.head(10))

folium.Choropleth(
    geo_data=geo_map_col,
    name="choropleth",
    data=dff,
    columns=["departame_cod", "inversion_millions"],
    key_on="properties.DPTO_CCDGO",#"feature.id",
    fill_color="YlGn",
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name="Inversion (millions)",
).add_to(m)

folium.LayerControl().add_to(m)

m